# Задание: получение котировок акций

Получите данные по котировкам акций со страницы:

mfd.ru/marketdata/?id=5&group=16&mode=3&sortHeader=name&sortOrder=1&selectedDate=01.11.2019

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from pylab import rcParams
rcParams['figure.figsize'] = 10, 4
plt.style.use('seaborn-whitegrid')

request = requests.get("https://mfd.ru/marketdata/?id=5&group=16&mode=3&sortHeader=name&sortOrder=1&selectedDate=01.11.2019")
html = BeautifulSoup(request.content)

In [2]:
data = []
table = html.find('table', {'id': 'marketDataList'})
tbody = table.find_all('tbody')[1]

for tr in tbody.find_all('tr'):
    data.append([td.get_text().strip() for td in tr.find_all('td')])

df = pd.DataFrame(
    data,
    columns=[
        'ticker',
        'time',
        'last_trade_price',
        'last_trade_price_delta',
        'last_trade_price_delta_in_percents',
        'previous_price',
        'open_price',
        'low_price',
        'high_price',
        'average_price',
        'day_volume',
        'day_money_volume',
        'trades_volume'
    ],
)


def to_float(value: str) -> float:
    if len(value) == 0:
        return 0
    
    modified = value.replace(' ', '').replace('N/A', '0').replace('%', '0')
    if modified[0] == '−':
        return -1 * float(modified[1:])
    elif modified[0] == '+':
        return float(modified[1:])
    else:
        return float(modified)


df['ticker'] = df['ticker'].str.replace('+', '', regex=False)
df['time'] = pd.to_datetime(df['time'], format='%d.%m.%Y')
df['last_trade_price'] = df['last_trade_price'].map(to_float)
df['last_trade_price_delta'] = df['last_trade_price_delta'].map(to_float)
df['last_trade_price_delta_in_percents'] = df['last_trade_price_delta_in_percents'].map(to_float)
df['previous_price'] = df['previous_price'].map(to_float)
df['open_price'] = df['open_price'].map(to_float)
df['low_price'] = df['low_price'].map(to_float)
df['high_price'] = df['high_price'].map(to_float)
df['average_price'] = df['average_price'].map(to_float)
df['day_volume'] = df['day_volume'].map(to_float)
df['day_money_volume'] = df['day_money_volume'].map(to_float)
df['trades_volume'] = df['trades_volume'].map(to_float)

> и найдите, по какому тикеру был максимальный рост числа сделок (в процентах) за 1 ноября 2019 года.

In [3]:
df[df['last_trade_price_delta_in_percents'] == df['last_trade_price_delta_in_percents'].max()]['ticker']

113    ИКРУСС-ИНВ
Name: ticker, dtype: object